In [7]:
# import section
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama.llms import OllamaLLM

In [8]:
# Load local pdf file exploding_kittens.pdf
file_path_local='sample/exploding_kittens.pdf'
loader = PDFPlumberLoader(file_path=file_path_local)
document = loader.load()
print("✅ Pdf file loaded successfully")

✅ Pdf file loaded successfully


In [9]:
# Chunking of data where chunk_size, chunk_overlap is configurble and defined as per data document & use case
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
data_chunks = text_splitter.split_documents(document)
print("✅ Data chunked successfully")

✅ Data chunked successfully


In [10]:
# Store chunks and embeddings received from ollama embedding model in chroma database
# For this example Embedding model is nomic-embed-text
Chroma.from_documents(data_chunks, OllamaEmbeddings(model="nomic-embed-text",show_progress=True),persist_directory="chroma_store", collection_name='local_rag')
print("✅ Data stored successfully in chroma db")

OllamaEmbeddings: 100%|██████████| 25/25 [00:56<00:00,  2.28s/it]

✅ Data stored successfully in chroma db


In [11]:
local_model = "deepseek-r1:1.5b"
embedding_model = "nomic-embed-text"

#load the chroma vector database created in previous step using same embeddings model.
vector_db = Chroma(persist_directory="chroma_store", embedding_function=OllamaEmbeddings(model=embedding_model), collection_name="local_rag")
question = "What are Defuse Cards?"
docs = vector_db.similarity_search(question, k=5)
print("Documents fetched from database : "+str(len(docs)))


Documents fetched from database : 5


In [ ]:
# Join all the documents fetched from database
context = "\n\n".join(doc.page_content for doc in docs)
# Create a RAG prompt in below format
conversation_prompt = f"""Answer the question based only on the following context. Be concise. If you cannot find the answer in the context, say "I cannot answer this based on the provided context."
Context: {context}
Question: {question}
Answer:
"""


# Call ollama chat api to generate the response from provided context
LANGUAGE_MODEL = OllamaLLM(model="deepseek-r1:1.5b")
response = LANGUAGE_MODEL.invoke(conversation_prompt)
print(response)
print("✅ All steps completed successfully")

<think>
Okay, so I need to figure out what the answer is for the question "What are Defuse Cards?" based on the given context. Let me read through the text again carefully.

Hmm, the context describes a card scenario in a game. It mentions something about Defuse Cards being the most powerful cards, used to stop an Exploding Kitten from dying. The description says that when someone's turn comes and they have a Defuse Card, if their card is "dead" (as in it just explodes but doesn't explode completely), then another player can end your turn by drawing a card.

Wait, actually, I think the text is explaining how to play the game, specifically about how to save oneself from an Exploding Kitten. So, the Defuse Card seems to be used to stop an explosion. It's not that you're saving others; it's that you're preventing another player from ending your turn.

So putting this together, Defuse Cards are cards that allow players to stop or end their turn by exploding a kitten. They prevent other pla